<a href="https://colab.research.google.com/github/KhondokerTanvirHossain/Ambiguous_Bangla_Word_NLP_PYTHON/blob/main/Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import codecs
#import heapq
from sklearn.utils import shuffle
import numpy as np
#from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
!pip install bnltk
from bnltk.stemmer import BanglaStemmer
#from nltk.stem import WordNetLemmatizer, PorterStemmer
#from bnlp.nltk_tokenizer import NLTK_Tokenizer
from bnltk.tokenize import Tokenizers
#from bnlp.bengali_word2vec import Bengali_Word2Vec


In [3]:
def tokenize(doc):
    tokens = []
    tokens = doc.split()
    return tokens


In [4]:
def tokenizeSent(doc):
    tokens = []
    tokens = doc.split('।')
    return tokens


In [29]:
from google.colab import files
uploaded = files.upload()
file_name_s = "stop.txt"
file_name_s = "extra.txt"

Saving stop.txt to stop (1).txt
Saving extra.txt to extra.txt


In [24]:
def tokenizeRemoveStop(doc):
    tokens = []
    tokens = doc.split()
    stopfile = uploaded[file_name_s].lower().decode("utf-8")
    stopwords = stopfile.read().lower()
    amb = codecs.open("extra.txt", 'r',  encoding="utf-8")
    #amb = []
    ambWords = amb.read().lower()
    for word in tokens:
        if word in stopwords:
            tokens.remove(word)
        if word in ambWords:
            tokens.remove(word)
    #print(tokens)
    return tokens


In [6]:
def bagOfWords(doc):
    words = tokenizeRemoveStopStem(doc)
    #print(words)
    sentences = tokenizeSent(doc)
    #most_freq = getFrequncies(words)
    word_freq = getFrequncies(words)
    #print(word_freq)
    bow = {}
    pos = 0
    for w in word_freq.keys():
        a = [0]*len(word_freq.keys())
        a[pos] = word_freq[w]
        bow[w] = a
        pos = pos + 1
    return bow


In [7]:
def getFrequncies(words):
    wordfreq = {}
    for i in words:
        if i not in wordfreq.keys():
            wordfreq[i] = 1
        else:
            wordfreq[i] += 1
    #most_freq = heapq.nlargest(50, wordfreq, key=wordfreq.get)
    return wordfreq


In [8]:
def tfidf(doc):
    words = tokenizeRemoveStopStem(doc)
    sentences = tokenizeSent(doc)
    most_freq = getFrequncies(words)
    word_idf_values = {}
    for i in words:
        doc_containing_word = 0
        for j in sentences:
            words_in_sentence = tokenizeRemoveStopStem(j)
            if i in words_in_sentence:
                doc_containing_word += 1
        word_idf_values[i] = np.log(len(sentences)/(1 + doc_containing_word))
    #return word_idf_values
    word_tf_values = {}
    for i in words:
        sent_tf_vector = []
        for j in sentences:
            doc_freq = 0
            for k in tokenizeRemoveStopStem(j):
                if i == k:
                    doc_freq += 1
            word_tf = doc_freq/(1 + len(tokenizeRemoveStopStem(j)))
            sent_tf_vector.append(word_tf)
        word_tf_values[i] = sent_tf_vector
    #return word_tf_values
    tfidf_values = []
    for i in word_tf_values.keys():
        tfidf_sentences = []
        for j in word_tf_values[i]:
            tf_idf_score = j * word_idf_values[i]
            tfidf_sentences.append(tf_idf_score)
        tfidf_values.append(tfidf_sentences)
    #print(tfidf_values)
    #return tfidf_values
    tf_idf_model = np.asarray(tfidf_values)
    tf_idf_model = np.transpose(tf_idf_model)
    return tf_idf_model


In [9]:
def removeStopStem(tokens):
    stopfile = codecs.open("stop.txt", 'r', "utf-8")
    stopwords = stopfile.read().lower()
    amb = codecs.open("extra.txt", 'r',  encoding="utf-8")
    ambWords = amb.read().lower()
    for word in tokens:
        if word in stopwords or word in ambWords:
            tokens.remove(word)
    stems = []
    bn_stemmer = BanglaStemmer()
    for i in tokens:
        stems.append(bn_stemmer.stem(i))
    return stems


In [10]:
def tokenizeRemoveStopStem(doc):
    #bnltk = NLTK_Tokenizer()
    #tokens = bnltk.word_tokenize(doc)
    t = Tokenizers()
    tokens = t.bn_word_tokenizer(doc)
    stopfile = codecs.open("stop.txt", 'r', "utf-8")
    stopwords = stopfile.read().lower()
    amb = codecs.open("extra.txt", 'r',  encoding="utf-8")
    ambWords = amb.read().lower()
    for word in tokens:
        if word in stopwords or word in ambWords:
            tokens.remove(word)
    stems = []
    bn_stemmer = BanglaStemmer()
    for i in tokens:
        stems.append(bn_stemmer.stem(i))
    return stems


In [11]:
#def getTrain(doc1, doc2):
#    tfidf1 = tfidf(doc1)
#    tfidf2 = tfidf(doc2)
#    a = np.concatenate((tfidf1, tfidf2))
#    return a


In [12]:
def getDataForTfidf(doc1, doc2, s):
    t = tfidf(s + doc1 + doc2)
    X = np.array(t)
    s1 = tokenizeSent(doc1)
    s2 = tokenizeSent(doc2)
    Y = []
    for i in s1:
        Y.append(0)
    for i in s2:
        Y.append(1)
    x = np.reshape(X[0], (1, len(X[0])))
    np.delete(X, 0, 0)
    return X, Y, x


In [13]:
def getDataForBow(doc1, doc2, s):
    bow = bagOfWords(doc1 + doc2 + s)
    #print(bow.keys())
    s1 = tokenizeRemoveStopStem(doc1)
    s2 = tokenizeRemoveStopStem(doc2)
    s3 = tokenizeRemoveStopStem(s)
    #s1 = addSimilarWords(s1)
    #s2 = addSimilarWords(s2)
    #s3 = addSimilarWords(s3)
    X = []
    Y = []
    pos = 0
    for i in bow.keys():
        if i in s1 and i not in s2:
            X.append(bow[i])
            Y.append(0)
        elif i in s2 and i not in s1:
            X.append(bow[i])
            Y.append(1)
        pos = pos + 1
    x = []
    pos = 0
    print(s3)
    for i in s3:
        if i in bow.keys():
            x.append(bow[i])
        pos = pos + 1
    return np.array(X),Y,x


In [32]:
from google.colab import files
uploaded = files.upload()
file_name1 = "কমলা লেবু.txt"
file_name2 = "কমলা বর্ণ.txt"


Saving কমলা লেবু.txt to কমলা লেবু (3).txt
Saving কমলা বর্ণ.txt to কমলা বর্ণ (3).txt


In [33]:
if __name__ == '__main__':
    sent2 = uploaded[file_name1].lower().decode("utf-8")
    sent1 = uploaded[file_name2].decode("utf-8")
    s = input()
    X, Y, x = getDataForBow(sent1, sent2, s)
    #print(X)
    X, Y = shuffle(X, Y)
    clf = MultinomialNB()
    clf.fit(X, Y)
    MultinomialNB()
    proba = clf.predict_proba(x)
    x1 = 0
    x2 = 0
    for i in range (len(proba)):
        x1 = x1 + proba[i][0]
        x2 = x2 + proba[i][1]
    if x1 >= x2 :
        print("komola fol")
    else:
        print("komola borno")

কমলা রঙ
['রঙ']
komola fol
